# Генерация новых уникальных transaction_id для bank_transactions.csv

Этот ноутбук загружает датасеты `clients.csv` и `bank_transactions.csv`, проверяет наличие дубликатов по столбцу `transaction_id` в `bank_transactions.csv`, генерирует новые уникальные `transaction_id` для всех строк и сохраняет обновленный датасет.

In [10]:
import pandas as pd
import uuid

# Загружаем датасеты
clients = pd.read_csv('clients.csv')
transactions = pd.read_csv('bank_transactions.csv')

# Выводим информацию о датасетах
print(f"Загружено {len(clients)} строк из clients.csv")
print(f"Загружено {len(transactions)} строк из bank_transactions.csv")

# Проверяем наличие столбца transaction_id
if 'transaction_id' not in transactions.columns:
    raise ValueError("Столбец 'transaction_id' отсутствует в bank_transactions.csv")

# Проверяем, что transaction_id не содержит пустых значений
if transactions['transaction_id'].isnull().any():
    print(f"Найдено {transactions['transaction_id'].isnull().sum()} пустых значений в transaction_id")
    transactions = transactions.dropna(subset=['transaction_id'])

Загружено 1200 строк из clients.csv
Загружено 4652 строк из bank_transactions.csv


## Проверка дубликатов в bank_transactions.csv

Проверяем наличие дубликатов по столбцу `transaction_id` и выводим статистику.

In [11]:
# Проверка дубликатов по transaction_id
duplicates = transactions[transactions['transaction_id'].duplicated(keep=False)]

if not duplicates.empty:
    unique_duplicated_ids = duplicates['transaction_id'].unique()
    print(f"Найдено {len(duplicates)} дубликатов по transaction_id.")
    print(f"Уникальных transaction_id с дубликатами: {len(unique_duplicated_ids)}")
    print("\nПримеры дублирующихся transaction_id:")
    
    # Выводим первые 5 дублирующихся transaction_id
    for tid in unique_duplicated_ids[:5]:
        print(f"transaction_id: {tid} (найдено {len(transactions[transactions['transaction_id'] == tid])} записей)")
    
    # Сохраняем дубликаты в файл для анализа
    duplicates.to_csv('duplicates_transactions.csv', index=False)
    print("\nДубликаты сохранены в /data/duplicates_transactions.csv")
else:
    print("Дубликаты по transaction_id не найдены.")

Найдено 1902 дубликатов по transaction_id.
Уникальных transaction_id с дубликатами: 145

Примеры дублирующихся transaction_id:
transaction_id: a5b6c7d8-e9f0-a1b2-c3d4-e5f6a7b8c9d0 (найдено 63 записей)
transaction_id: c7d8e9f0-a1b2-c3d4-e5f6-a7b8c9d0e1f2 (найдено 64 записей)
transaction_id: e9f0a1b2-c3d4-e5f6-a7b8-c9d0e1f2a3b4 (найдено 64 записей)
transaction_id: a1b2c3d4-e5f6-a7b8-c9d0-e1f2a3b4c5d6 (найдено 64 записей)
transaction_id: c3d4e5f6-a7b8-c9d0-e1f2-a3b4c5d6e7f8 (найдено 64 записей)

Дубликаты сохранены в /data/duplicates_transactions.csv


## Генерация новых уникальных transaction_id

Генерируем новые уникальные `transaction_id` в формате UUID для всех строк датасета и сохраняем обновленный датасет.

In [13]:
# Генерируем новые transaction_id для всех строк
transactions['transaction_id'] = [str(uuid.uuid4()) for _ in range(len(transactions))]

# Проверяем, остались ли дубликаты
new_duplicates = transactions[transactions['transaction_id'].duplicated(keep=False)]
if new_duplicates.empty:
    print("Новые transaction_id уникальны.")
else:
    print(f"Ошибка: найдено {len(new_duplicates)} дубликатов в новых transaction_id. Проверьте генерацию.")

# Выводим первые 5 строк для проверки
print("\nПервые 5 строк с новыми transaction_id:")
print(transactions[['transaction_id']].head())

# Сохраняем обновленный датасет
transactions.to_csv('bank_transactions_new_ids.csv', index=False)
print("Обновленный датасет сохранен в /data/bank_transactions_new_ids.csv")

Новые transaction_id уникальны.

Первые 5 строк с новыми transaction_id:
                         transaction_id
0  0ece3492-67f2-43cb-886e-5f99360439fa
1  eaef9e16-5429-44aa-bb06-f0f2834ded3a
2  ff9ae270-0906-420a-9a3c-ec4856e1cc2f
3  51854b88-558d-40fd-9b60-6e4c101c9964
4  9390ebae-7d5a-4216-ad68-c7b676852e5d
Обновленный датасет сохранен в /data/bank_transactions_new_ids.csv
